In [1]:
import pandas as pd 
import numpy as np 
from datetime import date
from optimisation_utiles import * 
df = pd.read_csv('ARP_Returns.csv')[:3026]
df_return = cal_return(df)
# My weekly change
df_weekly = df_return.resample('W-FRI').apply(lambda x: (1+x).prod()-1)

In [2]:
df_weekly = df_weekly[df_weekly.index>'2009-09-27']
df_weekly = df_weekly.drop(columns=['equity_momentum', 'equity_low_beta', 'equity_quality', 'equity_trend', 'fx_value', 'commodity_carry'])
df_return = df_return[df_return.index>'2009-09-27']
df_return = df_return.drop(columns=['equity_momentum', 'equity_low_beta', 'equity_quality', 'equity_trend', 'fx_value', 'commodity_carry'])


In [3]:
%%time
div_results = rolling_portfolio_optimisation(df_return=df_weekly, default_upper_bound=0.1, default_lower_bound=0, window_size=36, optimisation_freq=12, target='diversification_ratio', input_bounds=None, product='ARP')

2009-10-30 ---- 2012-10-28
2010-10-29 ---- 2013-10-27
2011-10-30 ---- 2014-10-31
2012-10-28 ---- 2015-10-30
2013-10-27 ---- 2016-10-30
2014-10-31 ---- 2017-10-29
2015-10-30 ---- 2018-10-28
2016-10-30 ---- 2019-10-27
Wall time: 8.65 s


In [4]:
input_bounds_max_sharpe = Duncans_weights(df_weekly, div_results)

2012-10-28
2013-10-27
2014-10-31
2015-10-30
2016-10-30
2017-10-29
2018-10-28
2019-10-27


In [5]:
# cap vol carry at 0.3
for date in input_bounds_max_sharpe:
    input_bounds_max_sharpe[date]['interest_rate_volatility_carry'] = [0, 0.03]
    input_bounds_max_sharpe[date]['commodity_volatility_carry'] = [0, 0.03]
    input_bounds_max_sharpe[date]['equity_volatility_carry'] = [0, 0.03]

In [14]:
sharpe_results = rolling_portfolio_optimisation(df_return=df_weekly, default_upper_bound=1, default_lower_bound=0, window_size=36, optimisation_freq=12, target='sharpe_ratio', input_bounds=input_bounds_max_sharpe)
df_component_weights_s, df_weights_s = generate_component_weights(df_weekly, sharpe_results)
df_component_weights_d, df_weights_d = generate_component_weights(df_weekly, div_results)

2009-10-30 ---- 2012-10-28
2010-10-29 ---- 2013-10-27
2011-10-30 ---- 2014-10-31
2012-10-28 ---- 2015-10-30
2013-10-27 ---- 2016-10-30
2014-10-31 ---- 2017-10-29
2015-10-30 ---- 2018-10-28
2016-10-30 ---- 2019-10-27


In [7]:
from PortfolioPerformance import * 
rebalance_results_s = Get_Portfolio_Index_Base_Case(df_return, df_component_weights_s, initial_notional=100)
rebalance_results_d = Get_Portfolio_Index_Base_Case(df_return, df_component_weights_d, initial_notional=100)

In [8]:
df_component_return_s = rebalance_results_s['PnL_df_net']
df_component_return_s = df_component_return_s[(df_component_return_s.index >= '2012-09-27')]
df_component_return_d = rebalance_results_d['PnL_df_net']
df_component_return_d = df_component_return_d[(df_component_return_d.index >= '2012-09-27')]

In [12]:
full_stats_d = save_results(div_results, rebalance_results_d, which='div')
full_stats_d

{'return': 0.6667234016757021,
 'volatility': 0.8975695419503791,
 'sharpe_ratio': 0.7428097439970406,
 'diversification_ratio': 3.475624601051589,
 'marginal_risk_contribution': array([0.09102891, 0.03018542, 0.07131181, 0.10026731, 0.0220006 ,
        0.07285618, 0.04074037, 0.00413493, 0.02765737, 0.04652486,
        0.09064276, 0.03101696, 0.0353399 , 0.01302052, 0.08120847,
        0.03053324, 0.10909996]),
 'component_risk_contribution': array([0.09102891, 0.03018542, 0.07131181, 0.10026731, 0.0220006 ,
        0.07285618, 0.04074037, 0.00413493, 0.02765737, 0.04652486,
        0.09064276, 0.03101696, 0.0353399 , 0.01302052, 0.08120847,
        0.03053324, 0.10909996]),
 'component_risk_contribution_pct': array([0.10141711, 0.03363017, 0.0794499 , 0.1117098 , 0.0245113 ,
        0.08117051, 0.04538965, 0.00460681, 0.03081363, 0.05183427,
        0.10098689, 0.0345566 , 0.03937288, 0.01450642, 0.09047596,
        0.03401768, 0.12155042]),
 'component_total_return_contribution': lo

In [13]:
full_stats_s = save_results(sharpe_results, rebalance_results_s, which='sharpe')
full_stats_s

{'return': 0.793641474349835,
 'volatility': 0.8413177156200677,
 'sharpe_ratio': 0.9433314663591811,
 'diversification_ratio': 3.436565357876944,
 'marginal_risk_contribution': array([0.10645125, 0.04268215, 0.0681958 , 0.07167255, 0.02438507,
        0.06543526, 0.05339436, 0.00532616, 0.01791171, 0.02753212,
        0.04301233, 0.03993928, 0.01703339, 0.04180043, 0.05122023,
        0.03982978, 0.12549585]),
 'component_risk_contribution': array([0.10645125, 0.04268215, 0.0681958 , 0.07167255, 0.02438507,
        0.06543526, 0.05339436, 0.00532616, 0.01791171, 0.02753212,
        0.04301233, 0.03993928, 0.01703339, 0.04180043, 0.05122023,
        0.03982978, 0.12549585]),
 'component_risk_contribution_pct': array([0.12652919, 0.0507325 , 0.08105832, 0.08519082, 0.02898438,
        0.0777771 , 0.06346516, 0.00633073, 0.02129007, 0.032725  ,
        0.05112495, 0.04747229, 0.02024608, 0.04968448, 0.06088096,
        0.04734214, 0.14916582]),
 'component_total_return_contribution': low